In [71]:
from pathlib import Path
import pandas as pd
import numpy as np 
# import pandas as pd
# import seaborn as sns

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
from tqdm.notebook import tqdm # to visualize the completion of a running loop
# meeans how values are chaning after each and every iteration
%matplotlib inline

In [72]:
# data_path = Path("../input")
test_base = pd.read_csv('baseline_locations_test.csv')
#path to our collected data, IMU dta
sub = pd.read_csv('sample_submission.csv')
# a sample file to read
sub.assign(
    latDeg = test_base.latDeg,
    lngDeg = test_base.lngDeg
).to_csv(
    'submission.csv', index=False
    # The index parameter is a boolean value 
    # that determines whether to include the index of the DataFrame in the CSV file
)

In [73]:
import numpy as np
from tqdm.notebook import tqdm

In [74]:
truths = ('ground_truth.csv')

df_list = []
cols = ['collectionName', 
        'sensorName', 
        'millisSinceGpsEpoch', 
        'latDeg',
        'lngDeg']

for t in tqdm(truths, total=10):
    df_sensor = pd.read_csv('baseline_locations_train.csv')  
    df_list.append(df_sensor)
df_truth = pd.concat(df_list, ignore_index=True)

df_basepreds = pd.read_csv('baseline_locations_train.csv')
df_all = df_truth.merge(df_basepreds,
                        how='inner', 
                        on=cols[:3], 
                        suffixes=('_truth', '_basepred'))
display(df_all[:5])


  0%|          | 0/10 [00:00<?, ?it/s]

,collectionName,sensorName,millisSinceGpsEpoch,latDeg_truth,lngDeg_truth,heightAboveWgs84EllipsoidM_truth,sensor_truth,latDeg_basepred,lngDeg_basepred,heightAboveWgs84EllipsoidM_basepred,sensor_basepred
0,2020-05-15-US-MTV-1,Sensor_s1,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1
1,2020-05-15-US-MTV-1,Sensor_s1,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1
2,2020-05-15-US-MTV-1,Sensor_s1,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1
3,2020-05-15-US-MTV-1,Sensor_s1,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1
4,2020-05-15-US-MTV-1,Sensor_s1,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Sensor_s1


In [75]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    #  read those 4 points first
    dlat = lat2 - lat1
    #  differnce between x1, x2
    dlon = lon2 - lon1
    #  differenece between y1, y2

    # haversine here
    #  The haversine formula is a very accurate way 
    # of computing distances between two points on the 
    # surface of a sphere using the latitude and longitude of the two points.
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [ ]:
df_all['error'] = calc_haversine(df_all.latDeg_truth, 
                                 df_all.lngDeg_truth, 
                                 df_all.latDeg_basepred,
                                 df_all.lngDeg_basepred)
   
    #  after haversine calculation
    #  calculate error percentage
print("Error percentage: ")

df_all.error.mean()
# accuracy = 100-error
err = df_all.error.mean()
print(err)
print("Accuracy: ")
print(100-err)



Error percentage: 
0.0
Accuracy: 
100.0
